requirements:
  bam2fastx
  minimap2
  cuteSV
  sniffles
  SURVIVOR
  mummer
  canu

In [ ]:
import os

data_folder = "/home/jupyter-ilya/vbcf/FTEST"
samples_bam = "m54067_210310_022505.demux.subreads.bam"

os.chdir(data_folder)

!ls {data_folder}

In [ ]:
path_to_minimap = "/home/jupyter-ilya/minimap2/minimap2" # /home/jupyter-ilya/minimap2/minimap2
path_to_sniffles = "/home/jupyter-ilya/Sniffles-master/bin/sniffles-core-1.0.12/sniffles" # /home/jupyter-ilya/Sniffles-master/bin/sniffles-core-1.0.12/sniffles
path_to_canu = "/home/jupyter-ilya/canu/build/bin/canu" # /home/jupyter-ilya/canu/build/bin/canu
path_to_SURVIVOR = "/home/jupyter-ilya/SURVIVOR/Debug/SURVIVOR" # /home/jupyter-ilya/SURVIVOR/Debug/SURVIVOR

SURVIVOR_settings = [1000, 2, 1, 1, 1, 30]

labels_file = open("labels", "r").read() # barcode_number ref.fasta annotation.gff\n
labels = {}
annot = {}

for i in labels_file.split("\n"):
    if i != "":
        pair = i.split(" ")
        labels[pair[0]] = pair[1]
        annot[pair[0]] = pair[2]
        

In [ ]:
!sudo bam2fastq {samples_bam} -o Answ --split-barcodes # Split data samples by barcodes

In [ ]:
for i in labels.keys(): # Unpacking .gz data
    !gunzip Answ.{i}_{i}.fastq.gz

In [ ]:
for i in labels.keys():
    !{path_to_minimap} -ax map-pb {labels[i]} Answ.{i}_{i}.fastq > Answ.{i}_{i}.{labels[i]}.alignment.sam # Align by reference
    !samtools view -b Answ.{i}_{i}.{labels[i]}.alignment.sam -o Answ.{i}_{i}.{labels[i]}.alignment.bam # Transform .sam to .bam
    !samtools sort Answ.{i}_{i}.{labels[i]}.alignment.bam -o Answ.{i}_{i}.{labels[i]}.mapped.sorted.bam # Sort subreads
    !samtools index Answ.{i}_{i}.{labels[i]}.mapped.sorted.bam # Index .bam
    
    

In [ ]:
tools_done = []

In [ ]:
# Sniffles part
!mkdir Sniffles_res

for i in labels.keys():
    !samtools calmd -E -u Answ.{i}_{i}.{labels[i]}.mapped.sorted.bam {labels[i]} > Answ.{i}_{i}.{labels[i]}.mapped.sorted.md.bam # Generate md tag for files
    !{path_to_sniffles} -m Answ.{i}_{i}.{labels[i]}.mapped.sorted.md.bam -v Sniffles_res/Answ.{i}_{i}.vcf

tools_done.append("Sniffles_res")

In [ ]:
# CuteSV part
!mkdir CuteSV_res
!mkdir CuteSV_res/work

for i in labels.keys():
    !cuteSV Answ.{i}_{i}.{labels[i]}.mapped.sorted.bam {labels[i]} CuteSV_res/Answ.{i}_{i}.vcf CuteSV_res/work
    
tools_done.append("CuteSV_res")

In [ ]:
# SURVIVOR part
vcfs = open("vcfs.txt", "w+")

for tool in tools_done:
    for i in labels.keys():
        vcfs.write(f"{tool}/Answ.{i}_{i}.vcf\n")
    
vcfs.close()

!{path_to_SURVIVOR} merge vcfs.txt {SURVIVOR_settings[0]} {SURVIVOR_settings[1]} {SURVIVOR_settings[2]} {SURVIVOR_settings[3]} {SURVIVOR_settings[4]} {SURVIVOR_settings[5]} Survivor_merged.vcf


In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox

# Mannualy filter mutations

class MutationWidget:
    def __init__(self, mutation, filepath):
        self.filepath = filepath
    
        self.button_accept = widgets.Button(description='Accept',disabled=False,)
        self.button_accept.on_click(self.accept_button_function)
        self.button_reject = widgets.Button(description='Reject',disabled=False,)
        self.button_reject.on_click(self.reject_button_function)
        self.mutation_string = mutation
        self.filepath = filepath

        self.out = widgets.Output()
        with self.out:
            print(mutation[:50])
    
        self.widget = HBox([self.button_accept, self.button_reject, self.out], layout={'border': '1px solid black'})
    
    def accept_button_function(self, button):
        # approved_list[index]=True
        self.button_accept.disabled = True
        self.button_reject.disabled = True
        self.button_accept.description='Accepted'
        self.button_accept.button_style='success'
        
        with open(self.filepath, "a") as f:
            f.write(self.mutation_string)
            #f.write('\n')
        
        
        # add_mutation_string to list of approved_mutations
    def reject_button_function(self, button):
        # approved_list[index]=False
        self.button_accept.disabled = True
        self.button_reject.disabled = True
        self.button_reject.description='Rejected'
        self.button_reject.button_style='danger'
        

file = open("Survivor_merged.vcf", "r")
widgets_list = []
for i in file:
    if i[:3] == "NC_":
        widgets_list += [MutationWidget(i, filepath = 'approved.vcf').widget]

VBox(widgets_list)

In [ ]:
# SNP's part
for i in labels.keys():
    !{path_to_canu} -p result -d canu_assembly{i} genomeSize=4.8m -pacbio-raw Answ.{i}_{i}.fastq
    !nucmer -mumreference -c 100 -p nucmer {labels[i]} canu_assembly{i}/result.contigs.fasta
    !show-snps -C nucmer.delta > nucmer{i}.snps

In [ ]:
# Annotate and extract mutated genes

def annotate(mutfile, annotation, output_name):
    snps = open(mutfile, "r").read().split("\n")

    mutations = []
    subst = [] 
    
    for i in snps[5:]:
        line = i.lstrip().split("  ")
        
        pos = line[0]
       
        mutations.append(int(pos))
        subst.append(line[:3])

#     for i in snps[5:]:
#         pos = i.split("  ")[0]
#         if pos != "":
#             mutations.append(int(pos))
#             subst.append(i.split("  ")[:3])
#         elif len(i.split("  "))>3:
#             mutations.append(int(i.split("  ")[1]))
#             subst.append(i.split("  ")[1:4])


    anno_file = open(annotation) # .gff
    annotation = anno_file.read().split('\n')
    
    starts = []
    badlines = []
    for gene in annotation:
        try:
            starts.append(int(gene.split('\t')[3]))
        except:
            badlines.append(gene)


    for line in badlines:
        annotation.remove(line)

    res = open(output_name, "w+")
    
    mutations.sort(reverse=True)
    for i in range(len(starts)-1, -1, -1):
        while mutations[0] >= starts[i]:
            res.write(annotation[i])
            res.write('\t')
            res.write('\t'.join(subst[0]))
            res.write('\n')
            
            del subst[0]
            del mutations[0]
            
            if len(mutations) == 0:
                break
        
        if len(mutations) == 0:
                break
            
#     for i in range(len(mutations)):
#         res.write(annotation[starts.index([x for x in starts if x <= mutations[i]][-1])])
#         res.write('\t')
#         res.write('\t'.join(subst[i]))
#         res.write('\n')

    res.close()
    

for i in labels.keys():
    annotate(f"nucmer{i}.snps", annot[i], f"muttations{i}.tsv")
